# Database Engineering

In [1]:
# Read in dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float, Date

In [2]:
# Create an engine to a SQLite database called 'wine.sqlite'
engine = create_engine("sqlite:///wine.sqlite")

In [3]:
# Create a connection to the engine
conn = engine.connect()

## Creating Table

In [4]:
# Use declarative_base to model the measurements 
Base = declarative_base()

class Reviews(Base):
    __tablename__ = 'reviews'
    
    id = Column(Integer, primary_key=True)
    country = Column(Text)
    description = Column(Text)
    designation = Column(Text)
    points = Column(Integer)
    price = Column(Integer)
    province = Column(Text)
    region_1 = Column(Text)
    region_2 = Column(Text)
    taster_name = Column(Text)
    taster_twitter_handle = Column(Text)
    title = Column(Text)
    variety = Column(Text)
    winery = Column(Text)

In [5]:
# Create the Reviews table in the database
Base.metadata.create_all(engine)

In [6]:
# Load the cleaned csv file 
df = pd.read_csv("raw_data/raw_data/wineData.csv", index_col=0, encoding='utf-8')
df.head(2)

country                                        description  \
0      US  Tart and snappy, the flavors of lime flesh and...   
1      US  Pineapple rind, lemon pith and orange blossom ...   

            designation  points  price  province             region_1  \
0                   NaN      87   14.0    Oregon    Willamette Valley   
1  Reserve Late Harvest      87   13.0  Michigan  Lake Michigan Shore   

            region_2         taster_name taster_twitter_handle  \
0  Willamette Valley        Paul Gregutt           @paulgwine    
1                NaN  Alexander Peartree                   NaN   

                                               title     variety      winery  
0      Rainstorm 2013 Pinot Gris (Willamette Valley)  Pinot Gris   Rainstorm  
1  St. Julian 2013 Reserve Late Harvest Riesling ...    Riesling  St. Julian

In [7]:
# Change wine_df to a dictionary
# orient='records' makes the dictionary list-like [{column -> value}]
data_reviews = df.to_dict(orient='records')

In [8]:
# Data is now a list of dictionaries that represent each row of data.
data_reviews[0]

{'country': 'US',
 'description': 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.',
 'designation': nan,
 'points': 87,
 'price': 14.0,
 'province': 'Oregon',
 'region_1': 'Willamette Valley',
 'region_2': 'Willamette Valley',
 'taster_name': 'Paul Gregutt',
 'taster_twitter_handle': '@paulgwine\xa0',
 'title': 'Rainstorm 2013 Pinot Gris (Willamette Valley)',
 'variety': 'Pinot Gris',
 'winery': 'Rainstorm'}

In [9]:
# Use MetaData from SQLAlchemy to reflect the tables
metadata = MetaData(bind=engine)
metadata.reflect()

In [10]:
# Save the referenct to the 'reviews' table
reviews_table = sqlalchemy.Table('reviews', metadata, autoload=True)

In [11]:
# Delete any pre-existing table
# DO NOT USE STEP IN PRODUCTION
conn.execute(reviews_table.delete())

In [12]:
# Insert data into the table
conn.execute(reviews_table.insert(), data_reviews)

In [13]:
# Test the first 5 rows
conn.execute('SELECT * FROM reviews LIMIT 5').fetchall()

[(1, 'US', 'Tart and snappy, the flavors of lime flesh and rind dominate. Some green pineapple pokes through, with crisp acidity underscoring the flavors. The wine was all stainless-steel fermented.', None, 87, 14, 'Oregon', 'Willamette Valley', 'Willamette Valley', 'Paul Gregutt', '@paulgwine\xa0', 'Rainstorm 2013 Pinot Gris (Willamette Valley)', 'Pinot Gris', 'Rainstorm'),
 (2, 'US', 'Pineapple rind, lemon pith and orange blossom start off the aromas. The palate is a bit more opulent, with notes of honey-drizzled guava and mango giving way to a slightly astringent, semidry finish.', 'Reserve Late Harvest', 87, 13, 'Michigan', 'Lake Michigan Shore', None, 'Alexander Peartree', None, 'St. Julian 2013 Reserve Late Harvest Riesling (Lake Michigan Shore)', 'Riesling', 'St. Julian'),
 (3, 'US', "Much like the regular bottling from 2012, this comes across as rather rough and tannic, with rustic, earthy, herbal characteristics. Nonetheless, if you think of it as a pleasantly unfussy country 